In [ ]:
import pandas as pd
import numpy as np

def normalize_feature(df):
    return df.apply(lambda column: (column - column.mean()) / column.std())


df = normalize_feature(pd.read_csv('data-3d.csv', names = ['square', 'bedrooms', 'price']))

ones = pd.DataFrame({'ones': np.ones(len(df))})
df = pd.concat([ones, df], axis = 1)

X_data = np.array(df[df.columns[0:3]])
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1)

print(X_data.shape, type(X_data))
print(y_data.shape, type(y_data))

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

alpha = 0.01
epoch = 500

with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, X_data.shape, name = 'X')
    y = tf.placeholder(tf.float32, y_data.shape, name = 'y')

with tf.name_scope('hypothesis'):
    W = tf.get_variable("weights", (X_data.shape[1], 1), initializer = tf.constant_initializer())
    y_pred = tf.matmul(X, W, name = 'y_pred')

with tf.name_scope('loss'):
    loss_op = 1 / (2 * len(X_data)) * tf.matmul(y_pred - y, y_pred - y, transpose_a = True)
    
with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate = alpha).minimize(loss_op)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./summary/linear-regression-3d', sess.graph)
    loss_data = []
    for e in range(1, epoch + 1):
        _, loss, w = sess.run([train_op, loss_op, W], feed_dict={X: X_data, y: y_data})
        loss_data.append(float(loss))
        if e % 100 == 0:
            log_str = "Epoch %d \t Loss=%.4g \t Model: y = %.4gx1 + %.4gx2 + %.4g"
            print(log_str % (e, loss, w[1], w[2], w[0]))

writer.close()    

In [ ]:
print(len(loss_data))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context = "notebook", style = "whitegrid", palette = "dark")

ax = sns.lineplot (x = 'epoch', y = 'loss', data = pd.DataFrame({'loss': loss_data, 'epoch': np.arange(epoch)}))
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
plt.show()